In [11]:
import email, glob
import tensorflow as tf
import pandas as pd
from flanker import mime
import string
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

# Reading from files Preprocessing
## Helper functions

In [12]:
def remove_content_in_braces(msg) :
	msg1 = ''
	cnt = 0
	for char in msg :
		if char == '{' :
			cnt += 1
		elif char == '}' :
			cnt -= 1
		elif cnt == 0 :
			msg1 += char
		else :
			continue
	return msg1

def remove_func_and_struct(msg) :
	msg1 = ''
	take_line = True
	msg = msg.splitlines()
	for line in msg :
		take_line = True
		if line == '' :
			continue
		words = line.split(' ')
		if words[0] == "func" :
			take_line = False
		elif words[0] == "type" :
			if len(words)  >= 3  and words[2] == "struct" :
				take_line = False
		if take_line :
			msg1 += (line + '\n')
	return msg1

def remove_other_code_lines(msg) :
	msg1 = ''
	take_line = True
	msg = msg.splitlines()
	i = 0	
	while i < len(msg) :
		if (msg[i] == '') or ("//" in msg[i]) :
			i += 1
			continue
		take_line = True
		line = msg[i]
		if "package" in line :
			words = line.split(' ')
			if len(words) < 4 :
				take_line = False
		elif "import" in line :
			words = line.split(' ')
			if len(words) < 4 :
				take_line = False
				if "(" in line :
					while ')' not in msg[i] :
						i += 1
		elif "const" in line :
			words = line.split(' ')
			if len(words) < 4 :
				take_line = False
				if "(" in line :
					while ')' not in msg[i] :
						i += 1
		if take_line :
			msg1 += line + '\n'
		i += 1
	return msg1


def remove_code(msg) :
	msg = (remove_content_in_braces(msg))
	msg = (remove_func_and_struct(msg))
	msg = (remove_other_code_lines(msg))
	return msg

def get_header(msg):
    msg = email.message_from_string(msg)
    mfrom = msg['From'].split('<')[0]
    return mfrom
    
def flan(msg):
    rt = ''
    msg = mime.from_string(msg)
    if msg.content_type.is_singlepart():
      temp = str(msg.body)
      temp = temp.splitlines()
      for _ in temp:
          if _.startswith('>'):
              continue
          elif _.startswith('On'):
              continue
          else:
              rt+=_+"\n"
    else :
      for part in msg.parts :
          if "(text/plain)" in str(part) :
              temp = str(part.body)
              temp = temp.splitlines()
              for _ in temp :
                  if _.startswith('>') :
                      continue
                  if _.startswith('On'):
                      continue
                  else :
                      rt+=_+"\n"
    return rt

## Reading files, tokenization & finding total_words 

In [16]:
rt =''
fpath = "/home/niki/Documents/BE_Project/my_EmailRecommmendation/features/Dataset/*/*.email"
files = glob.glob(fpath)
for file in files :
  f = open(file, "r")
  msg = f.read()
  msg = mime.from_string(msg)
  if msg.content_type.is_singlepart():
      temp = str(msg.body)
      temp = temp.splitlines()
      for _ in temp:
          if _.startswith('>'):
              continue
          elif _.startswith('On'):
              continue
          else:
              rt+=_+"\n"
  else :
      for part in msg.parts :
          if "(text/plain)" in str(part) :
              temp = str(part.body)
              temp = temp.splitlines()
              for _ in temp :
                  if _.startswith('>') :
                      continue
                  if _.startswith('On'):
                      continue
                  else :
                      rt+=_+"\n"
               
rt = remove_code(rt)
# print(rt)
rt = rt.split('\n')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(rt)
total_words = len(tokenizer.word_index) + 1
max_words=total_words

## Dataframe, apply functions

In [17]:
df = pd.DataFrame(columns=['body','replier', 'thread_no'])
users = []
fpath = "/home/niki/Documents/BE_Project/my_EmailRecommmendation/features/Dataset/*"
folder = glob.glob(fpath)
th_no = 0

for fol in folder:
    files = glob.glob(fol+'/*.email')
    flag = 0
    t = ''
    for file in files:
        if flag==0:
            data = open(file,'r')
            temp = data.read()
            header = get_header(temp)
            temp = flan(temp)
            temp = remove_code(temp)
            t = temp
            flag = 1
            users.append(header)
            
            continue
        data = open(file,'r')
        temp = data.read()
        header = get_header(temp)
        users.append(header)
        temp = flan(temp)
        temp = remove_code(temp)
        df = df.append({'body': t,'replier':header, 'thread_no':th_no}, ignore_index=True)
        t = t + temp
    th_no += 1

In [18]:
print(df.head)

<bound method NDFrame.head of                                                  body  \
0   I have client and server.\n--- Client\ntype\tA...   
1   I'm looking at the math package and I'm not se...   
2   I'm looking at the math package and I'm not se...   
3   once you get the whole request, you can then u...   
4   once you get the whole request, you can then u...   
5   i felt that this is necessary though i still f...   
6   This is true at the moment, but something we i...   
7   This is true at the moment, but something we i...   
8   This is true at the moment, but something we i...   
9   This is true at the moment, but something we i...   
10  This is true at the moment, but something we i...   
11  The following crude hack worked for me:\nApply...   
12  The following crude hack worked for me:\nApply...   
13  The following crude hack worked for me:\nApply...   
14  The following crude hack worked for me:\nApply...   
15  The following crude hack worked for me:\nApply...   
1

## Thread list no

In [19]:
h = df.replier
pprint(len(h))
h=list(h)

thread_no_list = df.thread_no
thread_no_list = list(thread_no_list)
print(thread_no_list)

68
[0, 1, 1, 2, 2, 3, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17]


## Encoding users

In [20]:
np.set_printoptions(threshold = sys.maxsize)
val = np.array(users)
w = open('one_hot.txt','w')

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(val)
print(integer_encoded)
user_indices = integer_encoded

one_hot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded),1)
one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
output_shape = one_hot_encoded.shape[1]
w.write(str(one_hot_encoded))
w.close()

[15 25 14 20 14 20 27 10 31 34 25 13 37  1  1  1 35 10 11  0 11 29 25  0
 24 21  5 17  5 21 33  6  1 33 37 33 31 26  2 24 12 37 33  6  7  6 12 26
  6  9 32 30 13 36 18 18  4  4 18 18 25 37 29 13  8 19  6 13  6  6 25 37
 25  3 37 37 25 22 22 26 22 22 16 23 28 20]


/home/niki/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Max no of users

In [21]:
print(user_indices)
input_weights = []
user_size = max(user_indices) + 1
print(user_size)


[15 25 14 20 14 20 27 10 31 34 25 13 37  1  1  1 35 10 11  0 11 29 25  0
 24 21  5 17  5 21 33  6  1 33 37 33 31 26  2 24 12 37 33  6  7  6 12 26
  6  9 32 30 13 36 18 18  4  4 18 18 25 37 29 13  8 19  6 13  6  6 25 37
 25  3 37 37 25 22 22 26 22 22 16 23 28 20]
38


# One hot encoding of user - User Vector (output)

In [22]:
onehot_encoded_final = []
for replier in h:
    idx1 = label_encoder.transform([replier])
    onehot = np.zeros(user_size)
    onehot[idx1] = 1
    onehot_encoded_final.append(list(onehot))
one_hot_encoded_final = np.array(onehot_encoded_final)
print(type(one_hot_encoded_final))

<class 'numpy.ndarray'>


# Binary encoding of users participating in each thread (concat vector)

In [23]:
index=0
weight_list = []
for i in range(0, max(thread_no_list)+1):
    temp_index=index
    array  = np.zeros(user_size)
    for j in range(temp_index, temp_index + thread_no_list.count(i)):
        array[user_indices[j]] += 1
        weight_list.append(list(array))
        index+=1

weights = np.array(weight_list)
pprint(weights) 

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.,

## Input 

In [11]:
x_train = df.body
# print(x_train)

## Find max length of body

In [12]:
def longest(l):
    m=0
    for k in l:
        m = max(len(k),m)
    return m
max_len = longest(x_train)

## Tokenization & conversion to sequence matrix

In [13]:
# max_words = 1294
# max_len = 3267
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
print(type(sequences_matrix))
print(len(sequences_matrix))

<class 'numpy.ndarray'>
68


In [14]:
b=np.zeros(user_size)
a=tf.convert_to_tensor(b)
#dense_cat = Dense(256, activation='relu')(a)
#flat1 = Dense(32, activation='relu')(dense_cat)

In [18]:
from keras.layers.merge import concatenate
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    inputs2 = Input(name='inputs2',shape=[user_size])
    layer2=Dense(256,name='FC2')(inputs2)
    
    merge=concatenate([layer,layer2])
    
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(output_shape,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=[inputs,inputs2],outputs=layer)
    return model

In [19]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 3182)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 3182, 50)          64700     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 38)                9766      
__________

In [20]:
model.fit([sequences_matrix,weights],one_hot_encoded_final,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 54 samples, validate on 14 samples
Epoch 1/10
54/54 [==============================] - 7s 124ms/step - loss: 0.6931 - acc: 0.5015 - val_loss: 0.6880 - val_acc: 0.9605
Epoch 2/10
54/54 [==============================] - 6s 106ms/step - loss: 0.6876 - acc: 0.8796 - val_loss: 0.6802 - val_acc: 0.9737
Epoch 3/10
54/54 [==============================] - 6s 103ms/step - loss: 0.6795 - acc: 0.9196 - val_loss: 0.6648 - val_acc: 0.9737
Epoch 4/10
54/54 [==============================] - 5s 84ms/step - loss: 0.6613 - acc: 0.9440 - val_loss: 0.6212 - val_acc: 0.9737
Epoch 5/10
54/54 [==============================] - 5s 98ms/step - loss: 0.6104 - acc: 0.9527 - val_loss: 0.3650 - val_acc: 0.9737
Epoch 6/10
54/54 [==============================] - 6s 105ms/step - loss: 0.3676 - acc: 0.9605 - val_loss: 0.1845 - val_acc: 0.9737
Epoch 7/10
54/54 [==============================] - 6s 107ms/step - loss: 0.2003 - acc: 0.9688 - val_loss: 0.1481 - val_acc: 0.9737
Epoch 8/10
54/54 [================